# Obtenez des données quotidiennes depuis le dépôt GitHub

Ce notebook récupère quotidiennement les données depuis la page web, en se basant sur la dernière date enregistrée dans le Lakehouse Bronze, afin d’extraire les données du jour suivant. Les données collectées sont ensuite enregistrées dans le Lakehouse Bronze.

In [ ]:
#Importation des bibliotèques
import requests
import io
import pandas as pd
from datetime import timedelta

In [ ]:
# Lien principal des fichiers CSV d'origine
base_url = "https://raw.githubusercontent.com/datasets-for-training/main/wind-power-dataset/"

# Chemin vers la table wind_power_production dans le Lakehouse Bronze
chemin_table_bronze = "abfss://WindPowerGeneration@onelake.dfs.fabric.microsoft.com/LH_Bronze.Lakehouse/Tables/wind_power_production"

In [ ]:
# Charge la table wind_power_production existante dans un DataFrame Pandas
df_spark = spark.read.format("delta").load(chemin_table_bronze)
df_pandas = df_spark.toPandas()

Display(df_spark)

Display(df_pandas)

In [ ]:
# Récupére la date la plus récente et le jour suivant
date_plus_recente = pd.to_datetime(df_pandas['date'], format = '%Y%m%d').max()
date_suivante = (date_plus_recente + timedelta(days = 1)).strftime('%Y%m%d')

Display(date_plus_recente)

# Créer l’URL et récupérer le fichier CSV du jour suivant
URL_fichier_suivant = f"{base_url}{date_suivante}_wind_power_production.csv"
contenu_fichier = requests.get(URL_fichier_suivant).content

Display(contenu_fichier)

# Charge le fichier CSV dans un DataFrame Pandas
df_pandas_nouveau = pd.read_csv(io.StringIO(contenu_fichier.decode('utf-8')))

Display(df_pandas_nouveau)

#Convertir la colonne de date au format datetime dans le DataFrame Pandas
df_pandas_nouveau['date'] = pd.to_datetime(df_pandas_nouveau['date'])

# Convertir le DataFrame Pandas en DataFrame Spark
df_spark_nouveau = spark.createDataFrame(df_pandas_nouveau, schema = df_spark.schema)

In [ ]:
# Insére les nouvelles données dans la table wind_power_production du Lakehouse Bronze
df_spark_nouveau.write.format("delta").mode("append").save(chemin_table_bronze)